<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;"><img src="https://pbs.twimg.com/profile_images/1187259618/unidata_logo_rgb_sm_400x400.png" alt="Unidata Logo" style="height: 98px;"></div>

<h1>Creating Animations</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="animation-image.png" alt="Example Animation" style="height: 300px;"></div>


## Overview:

* **Teaching:** 15 minutes
* **Exercises:** 15 minutes

### Questions
1. How are animations created with Matplotlib?
1. How can the data in a matplotlib plot be changed?
1. How can animations be displayed in the Jupyter Notebook?

### Objectives
1. <a href="#basicanimation">Create a basic animation with matplotlib</a>
1. <a href="#downloaddata">Download model output with Siphon</a>
1. <a href="#contour">Create an animated contour analysis of the output</a>

<a name="basicanimation"></a>
## 1. Create a basic animation
Matplotlib has powerful support for creating animated plots, though this requires going beyond the basics of simply calling functions to create plots. Instead, this requires knowing about Matplotlib's artists, and sometimes creating what are known as "callback functions"; these are functions that matplotlib can call at the appropriate time to perform the actual animation. In addition to creating interactive animations, Matplotlib has support for writing out these animations in a variety of formats; these include stand alone file such as animated GIFs and h.264 video files, as well as javascript-based animations suitable for the notebook.

Matplotlib has two classes to support creating animations: `ArtistAnimation` and `FuncAnimation`. The former controls animations by specifying a list of artists that should be drawn for each frame of the animation. The latter uses a user-specifed function to perform all of the updates for a given animation frame. `ArtistAnimation` is the simplest to use, but `FuncAnimation` is often more efficient.

We start below by doing the appropriate imports. The line setting `jshtml` tells matplotlib to display animations in the notebook by converting them to an HTML/Javascript output. Another option is `html5`, which tells matplotlib to render the animation as a video file compatible with the HTML5 `video` tag.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as manimation
plt.rcParams['animation.html'] = 'jshtml'

From here let's create a bunch of image plots:

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-5, 5)
for i in range(-5, 5):
    xvals = x - i
    bounds = (x.min(), x.max(), x.min(), x.max())
    im = ax.imshow(np.hypot(xvals, xvals[:, None]), extent=bounds, origin='lower',
                   norm=plt.Normalize(0, 15))

To facilitate making an animation, we need to do two things:
1. Create a list to hold a list of "things" to plot for each frame of the animation; in this case that list is just a single item, the image plot
2. Save the object created by `imshow` so we can use it in the animation

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-5, 5)

artists = []
for i in range(-5, 5):
    xvals = x - i
    bounds = (x.min(), x.max(), x.min(), x.max())
    im = ax.imshow(np.hypot(xvals, xvals[:, None]), extent=bounds, origin='lower',
                   norm=plt.Normalize(0, 15))
    artists.append([im]) # NOTE THE LIST

So now we have a list of list of artists. We can use this to create an animation:

In [ ]:
anim = manimation.ArtistAnimation(fig, artists, interval=500)
anim

Let's see now why we want a list of artists for each frame, by adding a text and a point on the plot:

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-5, 5)

artists = []
for i in range(-5, 5):
    xvals = x - i
    bounds = (x.min(), x.max(), x.min(), x.max())
    im = ax.imshow(np.hypot(xvals, xvals[:, None]), extent=bounds, origin='lower',
                   norm=plt.Normalize(0, 15))
    text = ax.text(0.5, 1.01, 'i: {}'.format(i), transform=ax.transAxes, ha='center')
    point, = ax.plot([i], [i], 'ro', zorder=2)
    artists.append([im, point, text])

In [ ]:
anim = manimation.ArtistAnimation(fig, artists, interval=500)
anim

Another way to accomplish this same task is to reset the data within a plot element rather than to recreate the plot. This uses `FuncAnimation`, as well as a function that is called for each frame, called a "callback function". First, let's make a single plot:

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-5, 5)

bounds = (x.min(), x.max(), x.min(), x.max())
im = ax.imshow(np.hypot(x, x[:, None]), extent=bounds, origin='lower',
               norm=plt.Normalize(0, 15))
text = ax.text(0.5, 1.01, 'i:', transform=ax.transAxes, ha='center')
point, = ax.plot([0], [0], 'ro', zorder=2)

Now we define the callback function. This function takes "framedata" that is used to change the frame. In this case, we'll define the frames using the same `range` call that we used to make the loop above. Instead of replotting, though, this function calls `set_data` and `set_text` to change the existing plot elements.

In [ ]:
def plot_frame(frame):
    xvals = x - frame
    im.set_data(np.hypot(xvals, xvals[:, None]))
    point.set_data([frame], [frame])
    text.set_text('i: {}'.format(frame))

anim = manimation.FuncAnimation(fig, plot_frame, frames=range(-5, 5), interval=500)
anim

In addition to being more efficient, the callback method can give you a lot more flexibility in terms of what's possible:

In [ ]:
# Taken from the matplotlib rain.py example by Nicolas P. Rougier

# Create new Figure and an Axes which fills it.
fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0, 0, 1, 1], frameon=False)
ax.set_xlim(0, 1), ax.set_xticks([])
ax.set_ylim(0, 1), ax.set_yticks([])

# Create rain data
n_drops = 50
rain_drops = np.zeros(n_drops, dtype=[('position', float, 2),
                                      ('size',     float, 1),
                                      ('growth',   float, 1),
                                      ('color',    float, 4)])

# Initialize the raindrops in random positions and with
# random growth rates.
rain_drops['position'] = np.random.uniform(0, 1, (n_drops, 2))
rain_drops['growth'] = np.random.uniform(50, 200, n_drops)

# Construct the scatter which we will update during animation
# as the raindrops develop.
scat = ax.scatter(rain_drops['position'][:, 0], rain_drops['position'][:, 1],
                  s=rain_drops['size'], lw=0.5, edgecolors=rain_drops['color'],
                  facecolors='none')

def update(frame_number):
    # Get an index which we can use to re-spawn the oldest raindrop.
    current_index = frame_number % n_drops

    # Make all colors more transparent as time progresses.
    rain_drops['color'][:, 3] -= 1.0/len(rain_drops)
    rain_drops['color'][:, 3] = np.clip(rain_drops['color'][:, 3], 0, 1)

    # Make all circles bigger.
    rain_drops['size'] += rain_drops['growth']

    # Pick a new position for oldest rain drop, resetting its size,
    # color and growth factor.
    rain_drops['position'][current_index] = np.random.uniform(0, 1, 2)
    rain_drops['size'][current_index] = 5
    rain_drops['color'][current_index] = (0, 0, 0, 1)
    rain_drops['growth'][current_index] = np.random.uniform(50, 200)

    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_edgecolors(rain_drops['color'])
    scat.set_sizes(rain_drops['size'])
    scat.set_offsets(rain_drops['position'])

# Construct the animation, using the update function as the animation
# director.
manimation.FuncAnimation(fig, update, interval=10)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="downloaddata"></a>
## 2. Download model data with Siphon

Now let's try animating some with some real world "data"--model output! First, let's the GFS "Best" time series from THREDDS using Siphon.

In [ ]:
from siphon.catalog import TDSCatalog

cat = TDSCatalog('http://thredds-test.unidata.ucar.edu/thredds/catalog/casestudies/irma/model/gfs/catalog.xml')
print(cat.datasets)

In [ ]:
best_ds = cat.datasets['Best GFS Half Degree Forecast Time Series']
ncss = best_ds.subset()

In [ ]:
from datetime import datetime

query = ncss.query().accept('netcdf4')
query.lonlat_box(west=-90, east=-55, south=15, north=30)
query.variables('Pressure_surface')
query.time_range(datetime(2017, 9, 6, 12), datetime(2017, 9, 11, 12))
nc = ncss.get_data(query)

In [ ]:
lon = nc.variables['longitude'][:]
lat = nc.variables['latitude'][:]
press = nc.variables['Pressure_surface']
print(press)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="contour"></a>
## 3. Plot animated contours

We have a [netCDF4-python](https://unidata.github.io/netcdf4-python/) dataset object now that contains all of the data for a given satellite image. We need to explore the variables available in the file and pull out the useful parts that we need to make a map.

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
from metpy.plots import add_metpy_logo

proj = ccrs.LambertConformal(central_longitude=-70)

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.coastlines()

# Add the MetPy Logo
add_metpy_logo(fig, x=15, y=15)

contours = np.arange(95000, 105000, 800)

artists = []
for time_slice in press:
    contourset = ax.contour(lon, lat, time_slice, contours, transform=ccrs.PlateCarree(),
                            colors='black')
    # contourset.collections is a *list* of all line collections from the contour call
    contour_artists = contourset.collections
    artists.append(contour_artists)

In [ ]:
anim = manimation.ArtistAnimation(fig, artists, interval=250)
anim

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Building on what was done above, produce an animation of GFS output. Add max wind speed
          ("Wind_speed_gust_surface") to the plot, as well as a timestamp.</li>
      <li>You are also free to experiment with different time ranges,
          geographic areas, or using different variables and plot types.</li>
    </ul>
</div>

In [ ]:
from netCDF4 import num2date

# Load data
cat = TDSCatalog('http://thredds-test.unidata.ucar.edu/thredds/catalog/casestudies/irma/model/gfs/catalog.xml')
best_ds = cat.datasets['Best GFS Half Degree Forecast Time Series']

# Access the best dataset using the subset service and request data
ncss = best_ds.subset()

# Set up query
query = ncss.query().accept('netcdf4')
query.lonlat_box(west=-90, east=-55, south=15, north=30)
query.variables('Pressure_surface', 'Wind_speed_gust_surface')
query.time_range(datetime(2017, 9, 6, 12), datetime(2017, 9, 11, 12))

# Pull useful pieces out of nc
nc = ncss.get_data(query)
lon = nc.variables['longitude'][:]
lat = nc.variables['latitude'][:]
press = nc.variables['Pressure_surface']
winds = nc.variables['Wind_speed_gust_surface']
time_var = nc.variables['time1']
times = num2date(time_var[:], time_var.units)

# Create a figure for plotting
proj = ccrs.LambertConformal(central_longitude=-70)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.coastlines()
add_metpy_logo(fig, x=15, y=15)

# Setup up the animation, looping over data to do the plotting that we want
pressure_levels = np.arange(95000, 105000, 800)
artists = []

#
# FILL THIS IN: Loop over all data and plot as appropriate
# HINT: Remember zip!
# 

# manimations.ArtistAnimation(...)

In [ ]:
# %load solutions/animation.py

<a href="#top">Top</a>
<hr style="height:2px;">